In [1]:
import pandas as pd
import psycopg2

conn = psycopg2.connect(database="postgres", user="postgres", password="password", host="localhost", port=5432)

In [2]:
df = pd.read_csv('sales_records.csv')
df

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Azerbaijan,Snacks,Online,C,10/8/14,535113847,10/23/14,934,152.58,97.44,142509.72,91008.96,51500.76
1,Central America and the Caribbean,Panama,Cosmetics,Offline,L,2/22/15,874708545,2/27/15,4551,437.20,263.33,1989697.20,1198414.83,791282.37
2,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Offline,M,12/9/15,854349935,1/18/16,9986,9.33,6.92,93169.38,69103.12,24066.26
3,Sub-Saharan Africa,Sao Tome and Principe,Personal Care,Online,M,9/17/14,892836844,10/12/14,9118,81.73,56.67,745214.14,516717.06,228497.08
4,Central America and the Caribbean,Belize,Household,Offline,H,2/4/10,129280602,3/5/10,5858,668.27,502.54,3914725.66,2943879.32,970846.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Sub-Saharan Africa,Niger,Cereal,Offline,L,8/26/12,836322486,9/11/12,5263,205.70,117.11,1082599.10,616349.93,466249.17
99996,Europe,Poland,Meat,Offline,C,12/3/13,110449349,12/10/13,3272,421.89,364.69,1380424.08,1193265.68,187158.40
99997,Sub-Saharan Africa,Comoros,Clothes,Online,M,8/7/13,193128764,8/31/13,9948,109.28,35.84,1087117.44,356536.32,730581.12
99998,Middle East and North Africa,Kuwait,Cosmetics,Online,L,6/28/11,701597058,7/3/11,7015,437.20,263.33,3066958.00,1847259.95,1219698.05


In [3]:
from psycopg2.extras import execute_values
region_list = df['Region'].unique().tolist()

cur = conn.cursor()
try:
    execute_values(
        cur,
        "INSERT INTO region (name) VALUES %s",
        [(region,) for region in region_list]
    )
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()

In [4]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM region")
    regions = [r for r in cur.fetchall()]
regions

[(1, 'Middle East and North Africa'),
 (2, 'Central America and the Caribbean'),
 (3, 'Sub-Saharan Africa'),
 (4, 'Europe'),
 (5, 'Asia'),
 (6, 'Australia and Oceania'),
 (7, 'North America')]

In [5]:
region_key_lookup = {region[1]: region[0] for region in regions}
country_map = {row['Country']: region_key_lookup[row['Region']] for index, row in df.iterrows()}
country_map

{'Azerbaijan': 1,
 'Panama': 2,
 'Sao Tome and Principe': 3,
 'Belize': 2,
 'Denmark': 4,
 'Germany': 4,
 'Turkey': 1,
 'United Kingdom': 4,
 'Kazakhstan': 5,
 'Haiti': 2,
 'Italy': 4,
 'Malta': 4,
 'Jordan': 1,
 'Cambodia': 5,
 'Saint Kitts and Nevis ': 2,
 'Cameroon': 3,
 'Bahrain': 1,
 'Solomon Islands': 6,
 'Monaco': 4,
 'Comoros': 3,
 'Iceland': 4,
 'Zambia': 3,
 'Egypt': 1,
 'Togo': 3,
 'Saudi Arabia': 1,
 'Morocco': 1,
 'Tunisia ': 1,
 'Angola': 3,
 'Vietnam': 5,
 'Belarus': 4,
 'Myanmar': 5,
 'Lithuania': 4,
 'Switzerland': 4,
 'Antigua and Barbuda ': 2,
 'Mongolia': 5,
 'Greenland': 7,
 'Eritrea': 3,
 'Saint Vincent and the Grenadines': 2,
 'Portugal': 4,
 'Somalia': 1,
 'Israel': 1,
 'Dominica': 2,
 'Luxembourg': 4,
 'Singapore': 5,
 'Cyprus': 4,
 'Botswana': 3,
 'Malawi': 3,
 'Kyrgyzstan': 5,
 'Macedonia': 4,
 'Rwanda': 3,
 'Cape Verde': 3,
 'Pakistan': 1,
 'Libya': 1,
 'Bangladesh': 5,
 'Benin': 3,
 'Burundi': 3,
 'China': 5,
 'Albania': 4,
 'Bulgaria': 4,
 'Central African

In [6]:
country_data = [(country, region_id) for country, region_id in country_map.items()]
country_data

[('Azerbaijan', 1),
 ('Panama', 2),
 ('Sao Tome and Principe', 3),
 ('Belize', 2),
 ('Denmark', 4),
 ('Germany', 4),
 ('Turkey', 1),
 ('United Kingdom', 4),
 ('Kazakhstan', 5),
 ('Haiti', 2),
 ('Italy', 4),
 ('Malta', 4),
 ('Jordan', 1),
 ('Cambodia', 5),
 ('Saint Kitts and Nevis ', 2),
 ('Cameroon', 3),
 ('Bahrain', 1),
 ('Solomon Islands', 6),
 ('Monaco', 4),
 ('Comoros', 3),
 ('Iceland', 4),
 ('Zambia', 3),
 ('Egypt', 1),
 ('Togo', 3),
 ('Saudi Arabia', 1),
 ('Morocco', 1),
 ('Tunisia ', 1),
 ('Angola', 3),
 ('Vietnam', 5),
 ('Belarus', 4),
 ('Myanmar', 5),
 ('Lithuania', 4),
 ('Switzerland', 4),
 ('Antigua and Barbuda ', 2),
 ('Mongolia', 5),
 ('Greenland', 7),
 ('Eritrea', 3),
 ('Saint Vincent and the Grenadines', 2),
 ('Portugal', 4),
 ('Somalia', 1),
 ('Israel', 1),
 ('Dominica', 2),
 ('Luxembourg', 4),
 ('Singapore', 5),
 ('Cyprus', 4),
 ('Botswana', 3),
 ('Malawi', 3),
 ('Kyrgyzstan', 5),
 ('Macedonia', 4),
 ('Rwanda', 3),
 ('Cape Verde', 3),
 ('Pakistan', 1),
 ('Libya', 1),
 

In [7]:
cur = conn.cursor()

try:
    execute_values(
        cur,
        "INSERT INTO country (name, region_id) VALUES %s",
        country_data
    )
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()


In [8]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM country")
    countries = {c[1]: c for c in cur.fetchall()}
countries

{'Azerbaijan': (1, 'Azerbaijan', 1),
 'Panama': (2, 'Panama', 2),
 'Sao Tome and Principe': (3, 'Sao Tome and Principe', 3),
 'Belize': (4, 'Belize', 2),
 'Denmark': (5, 'Denmark', 4),
 'Germany': (6, 'Germany', 4),
 'Turkey': (7, 'Turkey', 1),
 'United Kingdom': (8, 'United Kingdom', 4),
 'Kazakhstan': (9, 'Kazakhstan', 5),
 'Haiti': (10, 'Haiti', 2),
 'Italy': (11, 'Italy', 4),
 'Malta': (12, 'Malta', 4),
 'Jordan': (13, 'Jordan', 1),
 'Cambodia': (14, 'Cambodia', 5),
 'Saint Kitts and Nevis ': (15, 'Saint Kitts and Nevis ', 2),
 'Cameroon': (16, 'Cameroon', 3),
 'Bahrain': (17, 'Bahrain', 1),
 'Solomon Islands': (18, 'Solomon Islands', 6),
 'Monaco': (19, 'Monaco', 4),
 'Comoros': (20, 'Comoros', 3),
 'Iceland': (21, 'Iceland', 4),
 'Zambia': (22, 'Zambia', 3),
 'Egypt': (23, 'Egypt', 1),
 'Togo': (24, 'Togo', 3),
 'Saudi Arabia': (25, 'Saudi Arabia', 1),
 'Morocco': (26, 'Morocco', 1),
 'Tunisia ': (27, 'Tunisia ', 1),
 'Angola': (28, 'Angola', 3),
 'Vietnam': (29, 'Vietnam', 5),
 

In [9]:
type_list = df['Item Type'].unique().tolist()

cur = conn.cursor()
try:
    execute_values(
        cur,
        "INSERT INTO product_type (name) VALUES %s RETURNING id, name",
        [(t,) for t in type_list],
    )
    inserted_types = cur.fetchall()
    types = {t[1]: t[0] for t in inserted_types}
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()
types

{'Snacks': 1,
 'Cosmetics': 2,
 'Fruits': 3,
 'Personal Care': 4,
 'Household': 5,
 'Clothes': 6,
 'Vegetables': 7,
 'Office Supplies': 8,
 'Beverages': 9,
 'Meat': 10,
 'Cereal': 11,
 'Baby Food': 12}

In [10]:
# create dummy brand
cur = conn.cursor()
try:
    execute_values(
        cur,
        "INSERT INTO brand (name, logo, description) VALUES %s",
        [('dummy', '/assets/brand/logo.png', 'This is a placeholder brand for historical data.')]
    )
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()


In [11]:
from datetime import datetime
# create dummy user
cur = conn.cursor()
try:
    execute_values(
        cur,
        "INSERT INTO user_account (email, password_hash, created_at, last_login, is_admin) VALUES %s",
        [('test@test.com', 'asdfjghjkl', datetime.now().isoformat(), datetime.now().isoformat(), False)]
    )
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()


In [12]:
df[['Item Type', 'Unit Price', 'Unit Cost']].value_counts().reset_index(name='count')

,Item Type,Unit Price,Unit Cost,count
0,Office Supplies,651.21,524.96,8426
1,Cereal,205.70,117.11,8421
2,Baby Food,255.28,159.42,8407
3,Cosmetics,437.20,263.33,8370
4,Personal Care,81.73,56.67,8364
5,Meat,421.89,364.69,8320
6,Snacks,152.58,97.44,8308
7,Clothes,109.28,35.84,8304
8,Vegetables,154.06,90.93,8282
9,Household,668.27,502.54,8278



Every `Item Type`, `Unit Price`, `Unit Cost` are the same on each `Item Type`. So we can assume that these are the same products and should not duplicate them.


In [86]:
from random import randint
priority_map = {
    'L': 100,
    'M': 200,
    'H': 300,
    'C': 400
}
stock_values = [0,0,1,5,100,123,1000]
def serialize_order(row):
    # df variables: Item Type, Unit Price, Unit Cost

    product = {
        "name": "Dummy Product",
        "type": types[row['Item Type']],
        "brand": 1,
        "price": float(row['Unit Price']),
        "cost": float(row['Unit Cost']),
        "description": "N/A",
        "stock": stock_values[randint(0, len(stock_values) - 1)]
    }

    order = {
        "date": row["Order Date"],
        'ship_date': row['Ship Date'],
        'user_id': 1,
        'country_id': countries[row['Country']][0],
        'priority': priority_map[row['Order Priority']],
        'price':  float(row['Total Revenue']),
        'cost':   float(row['Total Cost']),
        'profit': float(row['Total Profit']),
    }

    order_item = {
        "order_id": None,
        "product_id": None,
        "amount": row["Units Sold"],
        "unit_price": row["Unit Price"]
    }
    return product, order, order_item

In [14]:
def get_or_create_product(product: dict):
    cur = conn.cursor()
    try:
        cur.execute(f"SELECT id FROM product WHERE price={product['price']} AND cost={product['cost']} AND type={product['type']} LIMIT 1")
        result = cur.fetchone()
        if result:
            conn.commit()
            return result[0]

        columns = ', '.join(product.keys())
        placeholders = ', '.join(['%s'] * len(product))
        
        cur.execute(
            f"INSERT INTO product ({columns}) VALUES ({placeholders}) RETURNING id",
            list(product.values())
        )
        product_id = cur.fetchone()[0]
        conn.commit()
        return product_id
    except Exception as e:
        conn.rollback()
        raise(e)
        

In [15]:
def create_order(order):
    cur = conn.cursor()
    try:
        columns = ', '.join(order.keys())
        placeholders = ', '.join(['%s'] * len(order))

        cur.execute(
            f"INSERT INTO orders ({columns}) VALUES ({placeholders}) RETURNING id",
            list(order.values())
        )
        order_id = cur.fetchone()[0]
        conn.commit()
        return order_id
    except Exception as e:
        conn.rollback()
        raise(e)
    

In [16]:
def create_order_item(order_item):
    cur = conn.cursor()
    try:
        columns = ', '.join(order_item.keys())
        placeholders = ', '.join(['%s'] * len(order_item))

        cur.execute(
            f"INSERT INTO order_item ({columns}) VALUES ({placeholders}) RETURNING id",
            list(order_item.values())
        )
        order_item_id = cur.fetchone()[0]
        conn.commit()
        return order_item_id
    except Exception as e:
        conn.rollback()
        raise(e)


In [17]:
from datetime import timedelta
success = 0
total = 100000
start_time = datetime.now()
for _, row in df.iterrows():
    p, o, oi = serialize_order(row.to_dict())
    p_id = get_or_create_product(p)
    o_id = create_order(o)
    oi['product_id'] = p_id
    oi['order_id'] = o_id
    oi_id = create_order_item(oi)
    success += 1
    if success % 500 == 0 and success > 0:
        current_time = datetime.now()
        elapsed_time = (current_time - start_time).total_seconds()
        progress = success / total
    
        if progress > 0:
            total_estimated_seconds = elapsed_time / progress
            remaining_seconds = total_estimated_seconds - elapsed_time
            remaining_time = str(timedelta(seconds=int(remaining_seconds)))
    
            print(f'Completed: {success} | '
                  f'Progress: {progress:.2%} | '
                  f'Time elapsed: {str(timedelta(seconds=int(elapsed_time)))} | '
                  f'Est. time remaining: {remaining_time}')
print(f'done')

Completed: 500 | Progress: 0.50% | Time elapsed: 0:00:03 | Est. time remaining: 0:10:29
Completed: 1000 | Progress: 1.00% | Time elapsed: 0:00:06 | Est. time remaining: 0:10:08
Completed: 1500 | Progress: 1.50% | Time elapsed: 0:00:09 | Est. time remaining: 0:10:10
Completed: 2000 | Progress: 2.00% | Time elapsed: 0:00:11 | Est. time remaining: 0:09:43
Completed: 2500 | Progress: 2.50% | Time elapsed: 0:00:14 | Est. time remaining: 0:09:16
Completed: 3000 | Progress: 3.00% | Time elapsed: 0:00:16 | Est. time remaining: 0:09:01
Completed: 3500 | Progress: 3.50% | Time elapsed: 0:00:19 | Est. time remaining: 0:08:51
Completed: 4000 | Progress: 4.00% | Time elapsed: 0:00:21 | Est. time remaining: 0:08:40
Completed: 4500 | Progress: 4.50% | Time elapsed: 0:00:24 | Est. time remaining: 0:08:33
Completed: 5000 | Progress: 5.00% | Time elapsed: 0:00:26 | Est. time remaining: 0:08:24
Completed: 5500 | Progress: 5.50% | Time elapsed: 0:00:29 | Est. time remaining: 0:08:18
Completed: 6000 | Prog

In [23]:
from faker import Faker
fake = Faker()

def fake_user():
    email = fake.email()
    passhash = hash(email)
    created = fake.date_time_this_year()
    last_login = fake.date_time_between_dates(created, datetime.now())
    return (email, passhash, created.isoformat(), last_login.isoformat())
fake_user()

('jordangary@example.com',
 1591980521699660303,
 '2025-01-25T10:30:49.572955',
 '2025-02-09T15:55:05.433939')

In [28]:
cur = conn.cursor()
try:
    execute_values(
        cur,
        "INSERT INTO user_account (email, password_hash, created_at, last_login) VALUES %s",
        [fake_user() for i in range(199)],
    )
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()

In [29]:
%%sql
SELECT COUNT(*) FROM user_account

,count
0,200


In [30]:
%%sql
SELECT COUNT(*) FROM product

,count
0,12


In [52]:

review_randoms = [sorted([max(1, min(5, randint(-3,9))) for i in range(7)]) for i in range(12)]
for l in review_randoms:
    print(f'{l} - avg:{sum(l)/len(l):02f}')


[3, 3, 4, 5, 5, 5, 5] - avg:4.285714
[1, 1, 2, 4, 5, 5, 5] - avg:3.285714
[1, 1, 1, 1, 1, 3, 3] - avg:1.571429
[1, 1, 1, 1, 5, 5, 5] - avg:2.714286
[1, 1, 3, 4, 5, 5, 5] - avg:3.428571
[1, 4, 5, 5, 5, 5, 5] - avg:4.285714
[1, 1, 3, 3, 5, 5, 5] - avg:3.285714
[1, 1, 1, 4, 5, 5, 5] - avg:3.142857
[1, 1, 1, 2, 3, 5, 5] - avg:2.571429
[1, 1, 1, 2, 4, 5, 5] - avg:2.714286
[1, 1, 1, 2, 5, 5, 5] - avg:2.857143
[1, 1, 1, 5, 5, 5, 5] - avg:3.285714


In [54]:
def fake_product_review(user_id, product_id):
    review_list = review_randoms[product_id - 1]
    
    return (
        user_id,
        product_id,
        review_list[randint(0, len(review_list) - 1)],
        fake.text(max_nb_chars=1024)
    )

In [62]:
def create_product_reviews(reviews):
    cur = conn.cursor()
    try:
        execute_values(
            cur,
            "INSERT INTO product_review (user_id, product_id, rating, review) VALUES %s",
            reviews
        )
        conn.commit()
    except Exception as e:
        print(e)
        conn.rollback()


In [70]:
for product_id in range(1, 13):
    reviews = [fake_product_review(i + 1, product_id) for i in range(200)]
    create_product_reviews(reviews)

In [71]:
%%sql
SELECT COUNT(*) FROM product_review;


,count
0,2400


In [79]:
%%sql
-- Product Review Aggregation
SELECT 
    p.id AS product_id,
    pt.name as product_type,
    p.name as product_name,
    COALESCE(AVG(pr.rating), 0) AS average_rating
FROM 
    product p
JOIN 
    product_review pr ON p.id = pr.product_id
JOIN
    product_type pt ON p.type = pt.id
GROUP BY 
    p.id, pt.name, p.name
ORDER BY 
    average_rating DESC;
--     p.id ASC;



,product_id,product_type,product_name,average_rating
0,6,Clothes,Dummy Product,4.345
1,1,Snacks,Dummy Product,4.285
2,5,Household,Dummy Product,3.495
3,2,Cosmetics,Dummy Product,3.380
4,7,Vegetables,Dummy Product,3.280
5,12,Baby Food,Dummy Product,3.220
6,8,Office Supplies,Dummy Product,3.205
7,11,Cereal,Dummy Product,2.995
8,4,Personal Care,Dummy Product,2.720
9,10,Meat,Dummy Product,2.600


In [81]:
%%sql
-- Total Sales by Country
SELECT 
    c.name AS country_name,
    TO_CHAR(o.date, 'YYYY-MM') AS order_month,
    SUM(o.price) AS total_sales,
    COUNT(o.id) AS order_count
FROM 
    orders o
JOIN 
    country c ON o.country_id = c.id
GROUP BY 
    c.name, TO_CHAR(o.date, 'YYYY-MM')
ORDER BY 
    c.name, order_month;


,country_name,order_month,total_sales,order_count
0,Azerbaijan,2010-01,2.047518e+08,157
1,Azerbaijan,2010-02,1.794266e+08,121
2,Azerbaijan,2010-03,1.638187e+08,133
3,Azerbaijan,2010-04,1.819031e+08,139
4,Azerbaijan,2010-05,1.719495e+08,119
...,...,...,...,...
632,Turkey,2017-03,2.478024e+07,21
633,Turkey,2017-04,4.766108e+07,37
634,Turkey,2017-05,9.846162e+06,17
635,Turkey,2017-06,4.070480e+07,23


In [93]:
df_sql1.pivot_table(
    index='order_month',
    columns='country_name',
    values='total_sales',
    fill_value=0
)

country_name,Azerbaijan,Belize,Denmark,Germany,Panama,Sao Tome and Principe,Turkey
order_month,,,,,,,
2010-01,2.047518e+08,3.593405e+08,1.940823e+08,1.219499e+08,1.462920e+08,3.619481e+08,42725684.40
2010-02,1.794266e+08,3.476981e+08,1.839257e+08,9.922286e+07,1.581915e+08,3.660615e+08,17381818.90
2010-03,1.638187e+08,3.532424e+08,2.111500e+08,1.272302e+08,1.472139e+08,3.633685e+08,20169601.30
2010-04,1.819031e+08,4.177550e+08,2.283509e+08,1.309267e+08,1.579042e+08,3.566669e+08,20467880.98
2010-05,1.719495e+08,3.219215e+08,2.122861e+08,1.133555e+08,1.376221e+08,4.079249e+08,38518528.54
...,...,...,...,...,...,...,...
2017-03,1.744848e+08,3.404760e+08,2.020492e+08,1.356433e+08,1.399185e+08,3.692322e+08,24780239.95
2017-04,2.130102e+08,3.632238e+08,1.936739e+08,1.005377e+08,1.812459e+08,4.002989e+08,47661078.15
2017-05,2.290722e+08,4.044962e+08,2.373231e+08,1.158848e+08,1.792625e+08,3.606095e+08,9846161.61


In [95]:
%%sql
-- Total Sales by Region
SELECT 
    r.name as region_name,
    TO_CHAR(o.date, 'YYYY-MM') AS order_month,
    SUM(o.price) AS total_sales,
    COUNT(o.id) AS order_count
FROM 
    orders o
JOIN 
    country c ON o.country_id = c.id
JOIN
    region r on c.region_id = r.id
GROUP BY 
    r.name, TO_CHAR(o.date, 'YYYY-MM')
ORDER BY 
    r.name, order_month;


,region_name,order_month,total_sales,order_count
0,Central America and the Caribbean,2010-01,5.056324e+08,392
1,Central America and the Caribbean,2010-02,5.058896e+08,379
2,Central America and the Caribbean,2010-03,5.004563e+08,393
3,Central America and the Caribbean,2010-04,5.756592e+08,394
4,Central America and the Caribbean,2010-05,4.595436e+08,356
...,...,...,...,...
359,Sub-Saharan Africa,2017-03,3.692322e+08,265
360,Sub-Saharan Africa,2017-04,4.002989e+08,275
361,Sub-Saharan Africa,2017-05,3.606095e+08,283
362,Sub-Saharan Africa,2017-06,4.276991e+08,296


In [96]:
df_sql_region.pivot_table(
    index='order_month',
    columns='region_name',
    values='total_sales',
    fill_value=0
)


region_name,Central America and the Caribbean,Europe,Middle East and North Africa,Sub-Saharan Africa
order_month,,,,
2010-01,5.056324e+08,3.160322e+08,2.474775e+08,3.619481e+08
2010-02,5.058896e+08,2.831486e+08,1.968084e+08,3.660615e+08
2010-03,5.004563e+08,3.383801e+08,1.839883e+08,3.633685e+08
2010-04,5.756592e+08,3.592776e+08,2.023710e+08,3.566669e+08
2010-05,4.595436e+08,3.256416e+08,2.104680e+08,4.079249e+08
...,...,...,...,...
2017-03,4.803946e+08,3.376925e+08,1.992651e+08,3.692322e+08
2017-04,5.444697e+08,2.942116e+08,2.606712e+08,4.002989e+08
2017-05,5.837587e+08,3.532080e+08,2.389184e+08,3.606095e+08


In [98]:
%%sql
-- Total Sales by Product
SELECT 
    p.id AS product_id,
    p.name AS product_name,
    TO_CHAR(o.date, 'YYYY-MM') AS order_month,
    SUM(oi.amount) AS units_sold,
    SUM(oi.amount * oi.unit_price) AS total_sales
FROM 
    product p
JOIN 
    order_item oi ON p.id = oi.product_id
JOIN 
    orders o ON oi.order_id = o.id
GROUP BY 
    p.id, p.name, TO_CHAR(o.date, 'YYYY-MM')
ORDER BY 
    p.name, order_month;


,product_id,product_name,order_month,units_sold,total_sales
0,10,Dummy Product,2010-01,262640,1.108052e+08
1,4,Dummy Product,2010-01,430810,3.521010e+07
2,8,Dummy Product,2010-01,562564,3.663473e+08
3,11,Dummy Product,2010-01,496213,1.020710e+08
4,5,Dummy Product,2010-01,429794,2.872184e+08
...,...,...,...,...,...
1087,7,Dummy Product,2017-07,355860,5.482379e+07
1088,4,Dummy Product,2017-07,428835,3.504868e+07
1089,10,Dummy Product,2017-07,365147,1.540519e+08
1090,6,Dummy Product,2017-07,449951,4.917065e+07


In [100]:
df_sql2.pivot_table(
    index='order_month',
    columns='product_id',
    values='total_sales'
)

product_id,1,2,3,4,5,6,7,8,9,10,11,12
order_month,,,,,,,,,,,,
2010-01,72845506.50,203572998.8,4787866.77,35210101.30,2.872184e+08,44040604.96,75345353.90,3.663473e+08,24287235.05,1.108052e+08,102071014.1,1.045586e+08
2010-02,62244705.84,184568789.2,3857049.99,30659620.09,2.913069e+08,50547791.84,80370637.04,2.941040e+08,20716100.60,1.426566e+08,91181050.4,9.969475e+07
2010-03,84957306.90,203071967.6,4520842.17,39753390.27,2.458218e+08,44527010.24,64752958.60,2.648933e+08,20371898.30,1.754084e+08,99334998.4,1.387794e+08
2010-04,64148751.66,222797120.0,4186333.68,32970045.46,3.118876e+08,47956544.48,55680211.14,3.481037e+08,21109128.95,1.732069e+08,86371167.3,1.255572e+08
2010-05,73513654.32,194722759.2,4627754.64,34264321.74,2.974009e+08,58454636.96,70084204.90,2.725887e+08,15789177.30,1.853856e+08,87341042.8,1.094053e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-03,69184501.98,177170490.8,3959418.75,43834986.47,2.787194e+08,45664177.92,84352163.68,2.580752e+08,23351378.70,2.136118e+08,85685569.2,1.029754e+08
2017-04,61109968.38,265132070.4,4020978.09,43046700.62,2.669084e+08,50912677.76,75679355.98,3.734500e+08,19496445.80,1.288304e+08,106184808.4,1.048795e+08
2017-05,54941006.40,184318710.8,4475619.66,36967786.68,3.268696e+08,49114693.92,75846357.02,3.365271e+08,25425987.60,2.110981e+08,109432605.7,1.214770e+08
